In [11]:
import os

In [12]:
%pwd

'/Users/nikki/Documents/ML Ops/MLOps-projects/datascienceProject'

In [13]:
os.chdir("../")
%pwd

'/Users/nikki/Documents/ML Ops/MLOps-projects'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [15]:
import ensure
print(dir(ensure))


['Check', 'Ensure', 'EnsureError', 'IntegerByteString', 'IntegerString', 'NumericByteString', 'NumericString', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_types', 'check', 'ensure', 'ensure_annotations', 'ensure_raises', 'ensure_raises_regex', 'main']


In [16]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

In [18]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [19]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [20]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/nikki/Documents/ML Ops/MLOps-projects/machinelearningpipeline/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/8g/sz3w79_54bgdwgjzjt0tn92h0000gn/T/ipykernel_48196/412457329.py", line 8, in <module>
    raise e
  File "/var/folders/8g/sz3w79_54bgdwgjzjt0tn92h0000gn/T/ipykernel_48196/412457329.py", line 2, in <module>
    config=ConfigurationManager()
  File "/var/folders/8g/sz3w79_54bgdwgjzjt0tn92h0000gn/T/ipykernel_48196/815227096.py", line 6, in __init__
    self.config=read_yaml(config_filepath)
  File "/Users/nikki/Documents/ML Ops/MLOps-projects/machinelearningpipeline/venv/lib/python3.10/site-packages/ensure/main.py", line 872, in __call__
    return_val = self.f(*args, **kwargs)
  File "/Users/nikki/Documents/ML Ops/MLOps-projects/datascienceProject/src/datascience/utils/common.py", line 36, in read_yaml
    raise 